In [25]:
import pandas as pd

for i in range (12):
  i = i+1
  if i!=1:
    df = pd.concat([df, pd.read_csv(f"file{i}.csv")])

  else:
    df = pd.read_csv("file1.csv")

In [26]:
df

,text,category,sub-category
0,Thank you for calling the Reserve Bank of Indi...,Legitimate,Legitimate
1,We are calling to provide information about th...,Legitimate,Legitimate
2,This is [Name] from the RBI. We are following ...,Legitimate,Legitimate
3,We have received your complaint about [bank]. ...,Legitimate,Legitimate
4,The RBI is conducting a survey about your expe...,Legitimate,Legitimate
...,...,...,...
521,We are calling to inform you that you have won...,LotteryScam,PrizeScam
522,This is a once-in-a-lifetime opportunity to cl...,LotteryScam,PrizeScam
523,Download TeamViewer so we can set up your remo...,LotteryScam,PrizeScam
524,Your computer is sending out suspicious signal...,LotteryScam,PrizeScam


In [28]:
df = df.drop("sub-category",axis = 1)
df

,text,category
0,Thank you for calling the Reserve Bank of Indi...,Legitimate
1,We are calling to provide information about th...,Legitimate
2,This is [Name] from the RBI. We are following ...,Legitimate
3,We have received your complaint about [bank]. ...,Legitimate
4,The RBI is conducting a survey about your expe...,Legitimate
...,...,...
521,We are calling to inform you that you have won...,LotteryScam
522,This is a once-in-a-lifetime opportunity to cl...,LotteryScam
523,Download TeamViewer so we can set up your remo...,LotteryScam
524,Your computer is sending out suspicious signal...,LotteryScam


In [33]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [34]:
def preprocess_text(text):

    text = text.lower()


    text = re.sub(r'[^a-z\s]', '', text)


    words = word_tokenize(text)


    words = [word for word in words if word not in stopwords.words('english')]


    return ' '.join(words)


In [37]:
df.text = df.text.apply(preprocess_text)

In [38]:
df.text

,text
0,thank calling reserve bank india help
1,calling provide information new banking regula...
2,name rbi following complaint regarding unautho...
3,received complaint bank investigating matter
4,rbi conducting survey experience banking servi...
...,...
521,calling inform brand new car pay processing fe...
522,onceinalifetime opportunity claim bumper prize...
523,download teamviewer set remote workstation sta...
524,computer sending suspicious signals needs imme...


In [39]:
X = df.text
y = df.category



In [43]:

arr = y.unique()

class_names_num = {i: arr[i] for i in range(len(arr))}
class_num_names =  {value: i for i, value in enumerate(arr)}
class_names_num

{0: 'Legitimate',
 1: 'GovernmentImpersonation',
 2: 'GovernmentImersonation',
 3: 'RemoteAccessRequest',
 4: 'BankingScam',
 5: 'legitemate',
 6: 'legitimate',
 7: 'Ecommerce Scam',
 8: 'LotteryScam'}

In [42]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
ps = WordNetLemmatizer()
cv = TfidfVectorizer(max_features=2500)



In [44]:
y = y.apply(lambda name: class_num_names[name])
y

,category
0,0
1,0
2,0
3,0
4,0
...,...
521,8
522,8
523,8
524,8


In [48]:
from sklearn.metrics import recall_score,classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,accuracy_score
def detect_model(corpos, data):
    x = cv.fit_transform(corpos).toarray()
    y = pd.get_dummies(data['category'])
    y = y.iloc[:, 1].values
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)
    fraud_detect = MultinomialNB().fit(x_train, y_train)
    print("model has trained.")
    y_ped = fraud_detect.predict(x_test)
    cong_m = confusion_matrix(y_test, y_ped)
    acc = accuracy_score(y_test, y_ped)
    recall = recall_score(y_test,y_ped)
    cl_r = classification_report(y_test,y_ped)
    print("Confusion matrix:", cong_m)
    print("Accuracy_score:", acc)
    print("recall_score is:",recall)
    print("Classification report id:",cl_r)
    return fraud_detect

In [49]:
model= detect_model(X, df)

model has trained.
Confusion matrix: [[613   1]
 [ 20  23]]
Accuracy_score: 0.9680365296803652
recall_score is: 0.5348837209302325
Classification report id:               precision    recall  f1-score   support

       False       0.97      1.00      0.98       614
        True       0.96      0.53      0.69        43

    accuracy                           0.97       657
   macro avg       0.96      0.77      0.83       657
weighted avg       0.97      0.97      0.96       657



In [58]:
from sklearn.metrics import recall_score, classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder

def detect_model(corpos, data, cv):

    x = cv.fit_transform(corpos).toarray()


    le = LabelEncoder()
    y = le.fit_transform(data['category'])


    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)


    model = MultinomialNB()
    model.fit(x_train, y_train)
    print("Model has trained.")


    y_pred = model.predict(x_test)


    conf_m = confusion_matrix(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    cl_r = classification_report(y_test, y_pred)

    print("Confusion Matrix:\n", conf_m)
    print("Accuracy Score:", acc)
    print("Recall Score:", recall)
    print("Classification Report:\n", cl_r)

    return model, le, cv


In [66]:
from sklearn.feature_extraction.text import CountVectorizer
tf = TfidfVectorizer(max_features=2500)
model2,le,cv2 = detect_model(X,df,cv)

Model has trained.
Confusion Matrix:
 [[115   0   0   0   0   0   0   0]
 [  5  37   1   0   0   0   0   0]
 [  4   0 100   0   0   0   0   0]
 [  1   0   4  59   0   1   0   5]
 [  1   0   0   0  45  13   0   0]
 [  1   0   0   0   3 104   0   0]
 [  0   0   0   0   0   0   0   9]
 [  3   0   0   0   0   0   0 146]]
Accuracy Score: 0.9223744292237442
Recall Score: 0.9223744292237442
Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94       115
           1       1.00      0.86      0.93        43
           3       0.95      0.96      0.96       104
           4       1.00      0.84      0.91        70
           5       0.94      0.76      0.84        59
           6       0.88      0.96      0.92       108
           7       0.00      0.00      0.00         9
           8       0.91      0.98      0.94       149

    accuracy                           0.92       657
   macro avg       0.82      0.80      0.81    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [67]:
def predict_category(model, cv, le, text):
    x = cv.transform([text]).toarray()

    y_pred = model.predict(x)[0]

    category_name = le.inverse_transform([y_pred])[0]

    return category_name


In [70]:
predict_category(model2,cv2,le,"We are from SBI you need to repay your loan")

'BankingScam'

In [71]:
predict_category(model2,cv2,le,"RBI calling")

'GovernmentImpersonation'

In [72]:
predict_category(model2,cv2,le,"We are calling to provide information about the new banking regulations regarding interest rates.")

'legitimate'

In [73]:

predict_category(model2,cv2,le,"RBI: Your application for a new cheque book has been approved.")

'Legitimate'

In [76]:
import joblib

# Save the model and vectorizer
joblib.dump(model2, "model.joblib")
joblib.dump(cv2, "tfidf_vectorizer.joblib")
joblib.dump(le,"labelencoder.joblib")

['labelencoder.joblib']